<a href="https://colab.research.google.com/github/dotsnangles/Transformers-Text-Classification/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q folium
!pip install -q transformers
!pip install -q datasets
!pip install -q huggingface_hub

     |████████████████████████████████| 4.2 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 60.9 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 51.4 MB/s 
     |████████████████████████████████| 346 kB 4.3 MB/s 
     |████████████████████████████████| 1.1 MB 45.3 MB/s 
     |████████████████████████████████| 212 kB 56.7 MB/s 
     |████████████████████████████████| 140 kB 75.1 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 127 kB 49.4 MB/s 
     |████████████████████████████████| 271 kB 57.8 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 144 kB 92.0 MB/s 
     |████████████████████████████████| 112 kB 75.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 re

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
import datasets

from huggingface_hub import notebook_login

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

import pandas as pd
import numpy as np

import os
import gc

In [3]:
# notebook_login()

xlm-roberta-base

klue/roberta-base

bert-base-multilingual-uncased

모델 3개
모델 당 케이폴드 5

### Download Models & Tokenizers

In [26]:
# models = ['klue/roberta-base', 'xlm-roberta-base', 'bert-base-multilingual-uncased']
# for adrs in models:
#     model = AutoModelForSequenceClassification.from_pretrained(adrs, num_labels=7)
#     tokenizer = AutoTokenizer.from_pretrained(adrs)

#     adrs.replace('/', '-')

#     tokenizer.save_pretrained(adrs)
#     model.save_pretrained(adrs)

### Data Load & K-Fold Interator Instantiation

In [4]:
!gdown -q 1K0v7liLb4ls5NMZrvrsEFQCfGqGduBSN

In [5]:
# df = pd.read_csv('/content/augmented_dataset.csv', index_col=False)
# df = df.drop(['Unnamed: 0'], axis=1)
# df.reset_index(inplace=True)
# X, y = df['title'], df['topic_idx']
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=100)
# splitgen = skf.split(X, y)

### Function Declaration

In [18]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [19]:
def tokenize(batch):
    return tokenizer(batch["title"], padding=True, truncation=True)

### Global Configration

In [6]:
num_labels = 7

In [27]:
models = ['klue/roberta-base', 'bert-base-multilingual-uncased', 'xlm-roberta-base']

In [13]:
data_path = '/content/train_data.csv'

In [17]:
batch_size = 64
num_train_epochs = 1

### Fine Tuning and Model Saving

In [29]:
for model_n in models:
    # Load Model & Tokenizer
    model_ckpt = model_n

    # Load Data & Prepare K-Fold
    df = pd.read_csv(data_path, index_col=False)
    # df = df.drop(['Unnamed: 0'], axis=1)
    # df.reset_index(inplace=True)
    X, y = df['title'], df['topic_idx']
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=100)
    splitgen = skf.split(X, y)

    print(model_ckpt)

    foldNum = 1
    for train_index, test_index in splitgen:

        print(f'{model_ckpt} {foldNum}-Fold')

        # Model instantiation
        model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels,)
        tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

        # Tokenization
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        train = {
            'title': X_train.to_list(),
            'topic_idx': y_train.to_list()
        }
        
        test =  {
            'title': X_test.to_list(),
            'topic_idx': y_test.to_list()
        }
        
        train = datasets.Dataset.from_dict(train)
        train_encoded = train.map(tokenize, batched=True, batch_size=None)
        train_encoded = train_encoded.rename_column('topic_idx', 'label')

        test = datasets.Dataset.from_dict(test)
        test_encoded = test.map(tokenize, batched=True, batch_size=None)
        test_encoded = test_encoded.rename_column('topic_idx', 'label')

        # Training 
        logging_steps = len(train_encoded) // batch_size
        
        model_ckpt = model_ckpt.replace('/', '-')
        model_name = f"{model_ckpt}-tc-{foldNum}"

        training_args = TrainingArguments(output_dir='log/' + model_name,
                                            num_train_epochs=num_train_epochs,
                                            learning_rate=2e-5,
                                            per_device_train_batch_size=batch_size,
                                            per_device_eval_batch_size=batch_size,
                                            weight_decay=0.01,
                                            evaluation_strategy="epoch",
                                            disable_tqdm=False,
                                            logging_steps=logging_steps,
                                            push_to_hub=False,
                                            log_level="error")

        trainer = Trainer(model=model,
                    args=training_args,
                    compute_metrics=compute_metrics,
                    train_dataset=train_encoded,
                    eval_dataset=test_encoded,
                    tokenizer=tokenizer,
                    data_collator=data_collator)
        
        trainer.train()

        tokenizer.save_pretrained(f'saved/{model_name}')
        model.save_pretrained(f'saved/{model_name}')

        model_ckpt = model_ckpt.replace('klue-roberta-base', 'klue/roberta-base')

        foldNum += 1

klue/roberta-base
klue/roberta-base 1-Fold


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.478500,0.337724,0.887745,0.886633


...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.650700,0.392557,0.872522,0.871187


RuntimeError: ignored

### Saved Models Validation

In [1]:
import gc
gc.collect()

3

In [2]:
path = '/content/saved/xlm-roberta-base-tc-1'

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(path)
print('-------------------------------------------------------------------------------------------')
print('-------------------------------------------------------------------------------------------')
tokenizer = AutoTokenizer.from_pretrained(path)

OSError: ignored

In [ ]:
trainer = Trainer(model=model)

In [62]:
data_test = datasets.load_dataset('csv', data_files='/content/test_data.csv')['train']

Using custom data configuration default-a09fe3c96d1d9d1e
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-a09fe3c96d1d9d1e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [63]:
data_test = data_test.remove_columns('index')

In [58]:
encoded_test = data_test.map(tokenize, batched=True, batch_size=None)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-a09fe3c96d1d9d1e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-e6ad7a2551b468c5.arrow


In [59]:
encoded_test = encoded_test.remove_columns('title')

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9131
})

In [60]:
preds_output = trainer.predict(encoded_test)

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: title. If title are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 9131
  Batch size = 8


In [53]:
preds_output.predictions

array([[ 2.4218233 , -0.25726327,  2.27026   , ..., -1.5357828 ,
        -1.3969632 , -1.6563606 ],
       [-0.975111  , -0.92582613, -0.02318902, ..., -0.6107239 ,
        -1.0220395 , -1.0928177 ],
       [ 0.8299036 ,  0.12403149,  3.8014946 , ..., -1.3463405 ,
        -1.9235437 , -0.3527752 ],
       ...,
       [-1.6523359 , -1.4514568 ,  3.4848905 , ..., -0.24669702,
        -1.8966044 , -0.08391794],
       [-0.11187033,  0.57389444,  3.7193058 , ..., -1.1683784 ,
        -1.818556  , -1.4992968 ],
       [ 1.3833963 ,  0.4802772 ,  2.543532  , ..., -1.8366673 ,
        -1.7923144 ,  1.4666952 ]], dtype=float32)